In [2]:
import os
import csv
import scanpy.api as sc


/home/kumara/miniconda3/envs/scganpytorch/lib/python3.6/site-packages/scanpy/api/__init__.py:6: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning


In [3]:
dataset_dir = "data/"
data_file = "matrix.mtx"
var_names_file = "genes.tsv"
obs_names_file = "barcodes.tsv"
output_h5ad_file = "68kPBMCs.h5ad"

data_path = os.path.join(dataset_dir,data_file)
var_names_path = os.path.join(dataset_dir,var_names_file)
obs_names_path = os.path.join(dataset_dir,obs_names_file)
output_h5ad_path = os.path.join(dataset_dir,output_h5ad_file)

In [4]:
with open(var_names_path, "r") as var_file:
    var_read = csv.reader(var_file, delimiter='\t')
    var_names = []
    for row in var_read:
        var_names.append(row[1])

In [5]:
with open(obs_names_path, "r") as obs_file:
    obs_read = csv.reader(obs_file, delimiter='\t')
    obs_names = []
    for row in obs_read:
        obs_names.append(row[0])

In [6]:
andata = sc.read(data_path) 
andata = andata.transpose()

In [7]:
andata

AnnData object with n_obs × n_vars = 68579 × 32738 

In [8]:
dir(andata)

['T',
 'X',
 '_BACKED_ATTRS',
 '_H5_ALIASES',
 '_H5_ALIASES_NAMES',
 '_X',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adata_ref',
 '_args_from_dict',
 '_boolean_index_to_array',
 '_check_boolean',
 '_check_dimensions',
 '_check_ellipsis',
 '_check_uniqueness',
 '_clean_up_old_format',
 '_gen_repr',
 '_get_and_delete_multicol_field',
 '_get_multicol_field_obs',
 '_get_multicol_field_var',
 '_get_obs_array',
 '_get_var_array',
 '_getitem_view',
 '_index_to_arrays',
 '_init_X_as_view',
 '_init_as_actual',
 '_init_as_view',
 '_inplace_subset_obs',
 '_inplace_subset_var',
 '_isview',
 '_laye

In [9]:
andata.var_names = var_names
andata.var_names_make_unique()
andata.obs_names = obs_names
andata.obs_names_make_unique()

andata.write(filename=output_h5ad_path)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [10]:
andata.X

<68579x32738 sparse matrix of type '<class 'numpy.float32'>'
	with 37323295 stored elements in Compressed Sparse Row format>

In [11]:
andata.data

/home/kumara/miniconda3/envs/scganpytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Use X instead of data, data will be removed in the future.
  """Entry point for launching an IPython kernel.


<68579x32738 sparse matrix of type '<class 'numpy.float32'>'
	with 37323295 stored elements in Compressed Sparse Row format>

In [12]:
andata.n_obs

68579

In [13]:
# filtering 

min_cells = 3,
min_genes = 10

sc_raw = andata

sc.pp.filter_cells(sc_raw, min_genes=min_genes, copy=False)
print("Filtering of the raw data is done with minimum "
      "%d cells per gene." % min_genes)

sc.pp.filter_genes(sc_raw, min_cells=min_cells, copy=False)
print("Filtering of the raw data is done with minimum"
      " %d genes per cell." % min_cells)

cells_count = sc_raw.shape[0]
genes_count = sc_raw.shape[1]
print("Cells number is %d , with %d genes per cell."
      % (cells_count, genes_count))

Filtering of the raw data is done with minimum 10 cells per gene.
Filtering of the raw data is done with minimum 3 genes per cell.
Cells number is 68579 , with 17789 genes per cell.


In [14]:
# scaling
scale = "normalize_per_cell_LS_20000"

if "normalize_per_cell_LS_" in str(scale):

    lib_size = int(scale.split('_')[-1])
    sc.pp.normalize_per_cell(sc_raw,
                             counts_per_cell_after=lib_size)
    scale = {"scaling": 'normalize_per_cell_LS',
                  "scale_value": lib_size}

else:

    warnings.warn("The scaling of the data is unknown, library size "
                  "library size normalization with 20k will be applied")

    lib_size = int(sscale.split('_')[-1])
    sc.pp.normalize_per_cell(sc_raw,
                             counts_per_cell_after=lib_size)
    self.scale = {"scaling": 'normalize_per_cell_LS',
                  "scale_value": lib_size}

print("Scaling of the data is done using " + scale["scaling"]
      + "with " + str(scale["scale_value"]))

Scaling of the data is done using normalize_per_cell_LSwith 20000


In [15]:
output_h5ad_processed_file = "68kPBMCs_processed.h5ad"
output_h5ad_processed_path = os.path.join(dataset_dir, output_h5ad_processed_file)
sc_raw.write(output_h5ad_processed_path)

In [26]:
import collections
import numpy as np

sct = collections.namedtuple('sc', ('barcode', 'count_no', 'genes_no'))

def process_line(line):

    scmd = sct(barcode=line.obs_names[0],
               count_no=int(np.sum(line.X)),
               genes_no=line.obs['n_genes'][0]
              )
    return line.X, scmd


for line in sc_raw:
    sc_genes, d = process_line(line)
    print(sc_genes, d)
    print(sc_genes.shape)

[ 0.        0.        0.       ...  0.       16.447369  0.      ] sc(barcode='AAACATACACCCAA-1', count_no=20000, genes_no=498)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATACCCCTCA-1', count_no=20000, genes_no=472)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATACCGGAGA-1', count_no=20000, genes_no=542)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATACTAACCG-1', count_no=20000, genes_no=349)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATACTCTTCA-1', count_no=19999, genes_no=446)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATACTGGATC-1', count_no=20000, genes_no=476)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATACTGTCTT-1', count_no=20000, genes_no=522)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATACTTCTAC-1', count_no=20000, genes_no=925)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATTGCTGCTC-1', count_no=20000, genes_no=489)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAACATTGCTTCGC-1', count_no=20000, genes_no=554)
(17789,)
[ 0.        0.    

[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACAGCACCCACCT-1', count_no=19999, genes_no=414)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACAGCACCTTGGA-1', count_no=20000, genes_no=419)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACAGCACGAACCT-1', count_no=20000, genes_no=490)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACAGCACGAGACG-1', count_no=20000, genes_no=779)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACAGCACGAGCTT-1', count_no=20000, genes_no=418)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACAGCACGTATGC-1', count_no=20000, genes_no=421)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACATATGCCAAGT-1', count_no=20000, genes_no=683)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACATATGCGGTAT-1', count_no=20000, genes_no=579)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACATATGCTCCCA-1', count_no=20000, genes_no=568)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACATATGGCATCA-1', count_no=20000, genes_no=664)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AACATATGGGGAGT-1', count

[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAGGCTTGTTCACT-1', count_no=20000, genes_no=366)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAGGCTTGTTCGCC-1', count_no=20000, genes_no=309)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAGGCTTGTTTCGT-1', count_no=20000, genes_no=1049)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAGGTCACAACCTG-1', count_no=19999, genes_no=676)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAGGTCACGAGCAG-1', count_no=20000, genes_no=359)
(17789,)
[ 0.        0.        0.       ...  0.       18.796993  0.      ] sc(barcode='AAGGTCACTCGATG-1', count_no=20000, genes_no=479)
(17789,)
[ 0.        0.        0.       ...  0.       16.949152  0.      ] sc(barcode='AAGGTCTGATCAGC-1', count_no=20000, genes_no=595)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAGGTCTGCGTCTC-1', count_no=20000, genes_no=498)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAGGTCTGCTGTCC-1', count_no=20000, genes_no=549)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='AAGGTCTGCTTTAC-1', count_no=20000

[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAAGGATCCCGT-1', count_no=20000, genes_no=464)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAAGGATCTGGA-1', count_no=20000, genes_no=544)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAATTGAAACAG-1', count_no=20000, genes_no=536)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAATTGACGGTT-1', count_no=20000, genes_no=466)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAATTGAGTCAC-1', count_no=20000, genes_no=603)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAATTGCAAGCT-1', count_no=20000, genes_no=440)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAATTGCCATAG-1', count_no=20000, genes_no=498)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAATTGGGTCAT-1', count_no=20000, genes_no=542)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAAATTGTCTCAT-1', count_no=20000, genes_no=473)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAACCGAACCTCC-1', count_no=20000, genes_no=467)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACAACCGAGAATGA-1', count

[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCACTGTTTGGG-1', count_no=20000, genes_no=576)
(17789,)
[ 0.        0.        0.       ...  0.        0.       12.870013] sc(barcode='ACCCAGCTACCTTT-1', count_no=20000, genes_no=599)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCAGCTATTGGC-1', count_no=20000, genes_no=502)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCAGCTCGCATA-1', count_no=20000, genes_no=543)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCAGCTCTCCCA-1', count_no=20000, genes_no=523)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCAGCTCTTCCG-1', count_no=19999, genes_no=530)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCAGCTGTTGTG-1', count_no=20000, genes_no=614)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCAGCTTCTTCA-1', count_no=20000, genes_no=502)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCGTACAAAGTG-1', count_no=19999, genes_no=492)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ACCCGTACCCACCT-1', count_no=19999, genes_no=386)
(17789,)
[0. 0. 0. ... 0. 0

...

[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGCGCTACGGTT-1', count_no=20000, genes_no=725)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGCGCTAGCACT-1', count_no=20000, genes_no=666)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGCGCTCTGAAC-1', count_no=19999, genes_no=558)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGCGCTTGTCGA-1', count_no=19999, genes_no=430)
(17789,)
[0.        0.        0.        ... 0.        7.9713035 0.       ] sc(barcode='ATCGCGCTTTCAGG-1', count_no=20000, genes_no=839)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGGAACCAGAGG-1', count_no=20000, genes_no=520)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGGAACCCCAAA-1', count_no=20000, genes_no=731)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGGAACCTCTAT-1', count_no=20000, genes_no=543)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGGAACGTCGAT-1', count_no=20000, genes_no=797)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATCGGAACTCCCGT-1', count_no=20000, genes_no=561)
(17789,)
[0. 0. 0. ... 0. 0

[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTAGATCCCGT-1', count_no=20000, genes_no=581)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTAGATCCGTC-1', count_no=20000, genes_no=597)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTCACATTCTC-1', count_no=20000, genes_no=558)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTCACCGTTAG-1', count_no=20000, genes_no=388)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTCACGCTAAC-1', count_no=20000, genes_no=505)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTCACTACTGG-1', count_no=19999, genes_no=776)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTGCTAACGTC-1', count_no=20000, genes_no=664)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTGCTACAGCT-1', count_no=19999, genes_no=725)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTGCTCGCAAT-1', count_no=19999, genes_no=651)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTGCTGCGAGA-1', count_no=20000, genes_no=287)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='ATGTTGCTGGTACT-1', count

[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATATGAATCGC-1', count_no=20000, genes_no=652)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATATGAGCACT-1', count_no=20000, genes_no=626)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATATGATCGGT-1', count_no=20000, genes_no=517)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATATGCCTATT-1', count_no=20000, genes_no=652)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATATGGAGCAG-1', count_no=20000, genes_no=512)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATATGGTCTAG-1', count_no=19999, genes_no=440)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATATGTGTTTC-1', count_no=20000, genes_no=605)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATTGAAGCTAC-1', count_no=20000, genes_no=538)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATTGAATTTCC-1', count_no=20000, genes_no=546)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATTGACGTGAT-1', count_no=20000, genes_no=1571)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CAAATTGAGTACCA-1', coun

[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATACTCAACCA-1', count_no=20000, genes_no=269)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATACTCGACTA-1', count_no=20000, genes_no=672)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATACTGCGATT-1', count_no=20000, genes_no=608)
(17789,)
[ 0.         0.         0.        ...  0.        13.6612015  0.       ] sc(barcode='CACATACTGTTTGG-1', count_no=19999, genes_no=541)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATACTTACAGC-1', count_no=20000, genes_no=573)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATACTTCATTC-1', count_no=20000, genes_no=525)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATACTTTAGGC-1', count_no=20000, genes_no=1265)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATACTTTCGGA-1', count_no=20000, genes_no=378)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATGGAACCACA-1', count_no=20000, genes_no=495)
(17789,)
[0. 0. 0. ... 0. 0. 0.] sc(barcode='CACATGGACCTACC-1', count_no=20000, genes_no=665)
(17789,)
[0. 0. 0. .

KeyboardInterrupt: 